In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.jit import ScriptModule, script_method, trace

from torchvision import datasets, transforms
from torch.autograd import Variable

cuda = True
batch_size = 256
test_batch_size = 100
epochs = 2
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 500

In [3]:
# Prepare dataset

cuda = cuda and torch.cuda.is_available()
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

transformers = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST('/home/hadim/mnist', train=True, download=True, transform=transformers)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

test_dataset = datasets.MNIST('/home/hadim/mnist', train=False, download=True, transform=transformers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True, **kwargs)

# Create the model

class Net(ScriptModule):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.is_training = torch.nn.Parameter(self.training)

    @script_method
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.is_training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

model = Net()
if cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

TypeError: _make_subclass(): argument 'data' (position 2) must be Tensor, not bool

In [15]:
# Define training and test functions

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            log_str = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
            print(log_str.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
            
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    log_str = '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
    print(log_str.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [16]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 5.599790


/home/hadim/local/conda/envs/nn/lib/python3.6/site-packages/torch/nn/functional.py:55: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 3.3783, Accuracy: 8565/10000 (85%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 4.687579

Test set: Average loss: 2.8952, Accuracy: 9003/10000 (90%)



In [19]:
# Run prediction
data, target = test_dataset[0]
data = data.expand([1] + list(data.size()))
data = data.cuda()
model(data)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)

In [20]:
# Save the model and weights
serial_model = torch.jit.trace(model.cpu(), torch.rand(1, 1, 28, 28))
serial_model.save("/home/hadim/mnist.model")

In [21]:
# Load the model and restore weights
model = torch.jit.load("/home/hadim/mnist.model")
mode = model.cuda()

# Run prediction
data, target = test_dataset[5]
data = data.expand([1] + list(data.size()))
data = data.cuda()

#model.cpu()
model(data)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)